In [2]:
import pandas as pd

In [3]:
dataset_name = 'amonia'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Prata

Emissões de Amônia (NH3) volatizado da agricultura

Unidade: Tonelada

In [3]:
df_prata = pd.DataFrame() # Criação de dataframe vazio para uso posterior

# Abas das tabelas na planilha "Emissões de amonia.xlsx"
tabelas = pd.ExcelFile(f'{raiz}/Bronze Emissões de amonia.xlsx').sheet_names

In [5]:
for NH3 in tabelas:
    df = (pd.read_excel(f'{raiz}/Bronze Emissões de amonia.xlsx', # Arquivo com resultados definitivos
                sheet_name=NH3,
                header=2,
                skiprows=lambda linha: linha in [3, 31, 32] # Linhas da tabela a serem ignoradas, se sobrepondo ao header
                )
                .merge(pd.read_parquet(f'{raiz}/prata_ibge_geocodigo.parquet') # Banco de dados com geocódigos
                        .filter(['UF-id', 'UF-nome'])
                        .drop_duplicates(), # Exclusão de duplicatas (geradas por ser um BD com municípios)
                        how='left',
                        left_on='Estado',
                        right_on='UF-nome'
                            )
                .drop(columns=['Estado', 'UF-nome']) # Exclusão de colunas desnecessárias para o prosseguimento
                # Derretimento da tabela, verticalizando-a
                .melt(
                    id_vars='UF-id', # Coluna a ser mantida
                    var_name='data', # Nome da nova coluna com os dados transformados em linhas
                    value_name='valor' # Nome da nova coluna com os valores das antigas colunas
                            )
                .query('valor != 0.0') # Exclusão de registros que possuam valor zerado
                .rename(columns={'UF-id':'geocodigo'})
                .assign(**{
                    'data': lambda df: pd.to_datetime(df['data'], format='%Y'), # Conversão de número inteiro para formato de data
                    'fonte': NH3[4:].capitalize(), # Criação de nova coluna com os títulos das tabelas cortando 'NH3' 
                        })
                )
    df_prata = (pd.concat([df_prata, df],
                         ignore_index=True)           
                )

df_prata = (df_prata.convert_dtypes()
            .sort_values(by=['geocodigo', 'data']) # Organização dos dados
            .set_index(['geocodigo', 'data', 'fonte'])
            )
df_prata.to_parquet(f'{raiz}/prata_amonia.parquet')

### Ouro

In [4]:
df = (
    pd.read_parquet(f'{raiz}/prata_amonia.parquet')
    .reset_index()
    .rename(columns={
        'geocodigo': 'state',
        'valor': 'value',
        'data': 'date',
        'fonte': 'label'
        })
    .assign(country = 76)
    .filter(['country', 'state', 'date', 'label', 'value'])
)

df.to_csv(f'{raiz}/ouro_amonia_agro.csv', index=False)

In [5]:
df

,country,state,date,label,value
0,76,11,1990-01-01,Fertilizante,42.584297
1,76,11,1990-01-01,Adubos organicos,1713.897024
2,76,11,1990-01-01,Deposição de dejetos,18329.172353
3,76,11,1990-01-01,Manejo de dejetos,5786.982625
4,76,11,1990-01-01,Queima resíduos de colheita,8.248944
...,...,...,...,...,...
4299,76,53,2021-01-01,Fertilizante,1831.43475
4300,76,53,2021-01-01,Adubos organicos,779.849122
4301,76,53,2021-01-01,Deposição de dejetos,873.594703
4302,76,53,2021-01-01,Manejo de dejetos,2642.492417
